<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/airflow/Airflow_Oozie_examples_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ETL процесс в Oozie определяется с помощью XML-файлов, которые описывают последовательность выполнения задач. Наш пример из workflow.xml (помимо него необходимы еще файлы, но в контесте нашего рассказа "хватит" и его):



In [ ]:
<workflow-app name="etl-workflow" xmlns="uri:oozie:workflow:0.5">
    <start to="extract-node" />

    <action name="extract-node">
        <shell>
            <job-tracker>${jobTracker}</job-tracker>
            <name-node>${nameNode}</name-node>
            <exec>${shellScriptExtract}</exec>
            <file>${shellScriptExtract}#extract.sh</file>
        </shell>
        <ok to="transform-node" />
        <error to="fail" />
    </action>

    <action name="transform-node">
        <shell>
            <job-tracker>${jobTracker}</job-tracker>
            <name-node>${nameNode}</name-node>
            <exec>${shellScriptTransform}</exec>
            <file>${shellScriptTransform}#transform.sh</file>
        </shell>
        <ok to="load-node" />
        <error to="fail" />
    </action>

    <action name="load-node">
        <shell>
            <job-tracker>${jobTracker}</job-tracker>
            <name-node>${nameNode}</name-node>
            <exec>${shellScriptLoad}</exec>
            <file>${shellScriptLoad}#load.sh</file>
        </shell>
        <ok to="end" />
        <error to="fail" />
    </action>

    <kill name="fail">
        <message>ETL process failed</message>
    </kill>

    <end name="end" />
</workflow-app>


И теперь посмотрим как это выглядит с использованием Airflow. ETL процесс в Airflow определяется с помощью Python-кода, который описывает DAG (Directed Acyclic Graph) - тот самый направленный ациклический граф.Тут файл будет называться условно etl_dag.py  и вот его содержимое:



In [ ]:
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.bash_operator import BashOperator

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2024, 7, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'etl_workflow',
    default_args=default_args,
    description='Example ETL workflow for DE students',
    schedule_interval=timedelta(days=1),
)

extract_task = BashOperator(
    task_id='extract',
    bash_command='sh /path/to/extract.sh',
    dag=dag,
)

transform_task = BashOperator(
    task_id='transform',
    bash_command='sh /path/to/transform.sh',
    dag=dag,
)

load_task = BashOperator(
    task_id='load',
    bash_command='sh /path/to/load.sh',
    dag=dag,
)

extract_task >> transform_task >> load_task


Просто сравните "визуальную" составляющую. Очевидно, что Airflow даже на этапе описания тех или иных процессов выглядит удобнее, понятнее и аккуратнее. Не говоря уже об интерфейсе).

Apache Airflow предоставляет большую гибкость и удобство использования по сравнению с Apache Oozie, особенно для разработчиков, которые предпочитают использовать Python. В то время как Oozie все еще полезен для интеграции с экосистемой Hadoop, Airflow выигрывает в плане масштабируемости, расширяемости и поддержки сообщества.

